In [1]:
# run this to shorten the data import from the files
import os
cwd = os.path.dirname(os.getcwd())+'/'
path_data = os.path.join(os.path.dirname(os.getcwd()), 'datasets/')


In [2]:
import torch
import torch.nn as nn

data = 'The rabbit-hole went straight on like a tunnel for some way, and then dipped suddenly down, so suddenly that Alice had not a moment to think about stopping herself before she found herself falling down a very deep well.'

chars = ['r', 'l', 'n', 'k', 's', 'c', 'T', 'd', 'y', ',', 'o', '.', 'u', '-', 'h', 'f', 'g', 'm', 'a', 'e', 'b', 'p', 'v', 'i', ' ', 'A', 't', 'w']

In [3]:
# exercise 01

"""
Creating a RNN model for text generation

At PyBooks, you've been tasked to develop an algorithm that can perform text generation. The project involves auto-completion of book names. To kickstart this project, you decide to experiment with a Recurrent Neural Network (RNN). This way, you can understand the nuances of RNNs before moving to more complex models.

The following has been imported for you: torch, torch.nn as nn.

The data variable has been initialized with an excerpt from Alice's Adventures in Wonderland by Lewis Carroll.
"""

# Instructions

"""

    Include an RNN layer and linear layer in RNNmodel class

    Instantiate the RNN model with input size as length of chars, hidden size of 16, and output size as length of chars.


"""

# solution

# Include an RNN layer and linear layer in RNNmodel class
class RNNmodel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNmodel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
      h0 = torch.zeros(1, x.size(0), self.hidden_size)
      out, _ = self.rnn(x, h0)  
      out = self.fc(out[:, -1, :])  
      return out

# Instantiate the RNN model
model = RNNmodel(len(chars), 16, len(chars))

#----------------------------------#

# Conclusion

"""
Great job! By successfully creating this RNN model, you have taken the first step towards developing an advanced text generation system. By adding the fully connected layer for text generation, you have allowed RNN to predict the next element in a sequence. Let's further train and evaluate the model. Keep going!
"""

"\nGreat job! By successfully creating this RNN model, you have taken the first step towards developing an advanced text generation system. By adding the fully connected layer for text generation, you have allowed RNN to predict the next element in a sequence. Let's further train and evaluate the model. Keep going!\n"

In [1]:
# exercise 02

"""
Text generation using RNN - Training and Generation

The team at PyBooks now wants you to train and test the RNN model, which is designed to predict the next character in the sequence based on the provided input for auto-completion of book names. This project will help the team further develop models for text completion.

The model instance for the RNNmodel class is preloaded for you. The data variable has been preprocessed and encoded as a sequence.

The inputs and targets variable are preloaded for you.
"""

# Instructions

"""

    Instantiate the loss function which will be used to compute the error of our model.

    Instantiate the optimizer from PyTorch's optimization module.

    Run the model training process by setting the model to the train mode and zeroing the gradients before performing an optimization step.

    After the training process, switch the model to evaluation mode to test it on a sample input.

"""

# solution

# Instantiate the loss function
criterion = nn.CrossEntropyLoss()
# Instantiate the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
for epoch in range(100):
    model.train()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/100, Loss: {loss.item()}')

# Test the model
model.eval()
test_input = char_to_ix['r']
test_input = nn.functional.one_hot(torch.tensor(test_input).view(-1, 1), num_classes=len(chars)).float()
predicted_output = model(test_input)
predicted_char_ix = torch.argmax(predicted_output, 1).item()
print(f"Test Input: 'r', Predicted Output: '{ix_to_char[predicted_char_ix]}'")

#----------------------------------#

# Conclusion

"""
Well done! Your RNN model has been successfully trained and tested. The model looks to be predicting the word 'rabbit'. You can also explore the same example with LSTMs or GRUs.
"""

'\n\n'

In [2]:
# exercise 03

"""
Building a generator and discriminator

At PyBooks, you're tasked with working on an automatic text generator to help writers overcome writer's block. By using GANs, or Generative Adversarial Networks, you believe you can create a system where one network, the generator, creates new text while the other network, the discriminator, evaluates its authenticity. To do this, you need to initialize both a generator and discriminator network. These networks will then be trained against each other to create new, believable text.

The following has been imported for you: torch, torch.nn as nn.
"""

# Instructions

"""

    Define the Generator class with a linear layer for sequential data and a sigmoid activation function.

    Pass the input through the defined model in the forward() method of the Generator class.

    Define a Discriminator class with the same layers and activation function, taking care when defining the dimensions.

"""

# solution

# Define the generator class
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(nn.Linear(seq_length, seq_length), nn.Sigmoid())
    def forward(self, x):
        return self.model(x)

# Define the discriminator networks
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(nn.Linear(seq_length, 1), nn.Sigmoid())
    def forward(self, x):
        return self.model(x)

#----------------------------------#

# Conclusion

"""
Great work! You have successfully defined a generator and a discriminator for your GAN. These networks will be the backbone of your text generation system. Now you're ready to move on to the next step, which is to train these networks against each other so they can start generating and evaluating new text!
"""

'\n\n'

In [3]:
# exercise 04

"""
Training a GAN model

Your team at PyBooks has made good progress in building the text generator using a Generative Adversarial Network (GAN). You have successfully defined the generator and discriminator networks. Now, it's time to train them. The final step is to generate some fake data and compare it with the real data to see how well your GAN has learned. We have used tensors as an input and the output would try to resemble the input tensors. The team at PyBooks can then use this synthetic data for text analysis as the features will have same relationship as text data.

The generator and discriminator have been initialized and saved to generator and discriminator, respectively.

The following variables have been initialized in the exercise:

    seq_length = 5: Length of each synthetic data sequence
    num_sequences = 100: Total number of sequences generated
    num_epochs = 50: Number of complete passes through the dataset
    print_every = 10: Output display frequency, showing results every 10 epochs

"""

# Instructions

"""

    Define the loss function for binary classification and the Adam optimizer.

    Train the discriminator by unsqueezing real_data and preventing gradient recalculations.
---

    Train the generator by calculating the loss and zeroing the gradients.
---

    Detach the tensor to prevent further computation and print data.

"""

# solution

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer_gen = torch.optim.Adam(generator.parameters(), lr=0.001)
optimizer_disc = torch.optim.Adam(discriminator.parameters(), lr=0.001)

for epoch in range(num_epochs):
    for real_data in data:
        # Unsqueezing real_data and prevent gradient recalculations
        real_data = real_data.unsqueeze(0)
        noise = torch.rand((1, seq_length))
        fake_data = generator(noise)
        disc_real = discriminator(real_data)
        disc_fake = discriminator(fake_data.detach())
        loss_disc = criterion(disc_real, torch.ones_like(disc_real)) + criterion(disc_fake, torch.zeros_like(disc_fake))
        optimizer_disc.zero_grad()
        loss_disc.backward()
        optimizer_disc.step()

        # Train the generator
        disc_fake = discriminator(fake_data)
        loss_gen = criterion(disc_fake, torch.ones_like(disc_fake))
        optimizer_gen.zero_grad()
        loss_gen.backward()
        optimizer_gen.step()

    if (epoch+1) % print_every == 0:
        print(f"Epoch {epoch+1}/{num_epochs}:\t Generator loss: {loss_gen.item()}\t Discriminator loss: {loss_disc.item()}")

print("\nReal data: ")
print(data[:5])

print("\nGenerated data: ")
for _ in range(5):
    noise = torch.rand((1, seq_length))
    generated_data = generator(noise)
    # Detach the tensor and print data
    print(torch.round(generated_data).detach())

#----------------------------------#

# Conclusion

"""
Well done! You have now successfully generated and printed synthetic data using the trained generator network. Fantastic work! The team at PyBooks can then use this synthetic data for text analysis.
"""

'\n\n'

In [4]:
# exercise 05

"""
Text completion with pre-trained GPT-2 models

Back at PyBooks, your current project involves creating captivating narratives based on existing stories to engage customers and enhance their experience. To achieve this, you need a powerful text generation tool that can seamlessly generate compelling text continuations. You'll be using a pre-trained model to get the job done.

The following has been loaded for you: torch and GPT2Tokenizer,GPT2LMHeadModel from transformers.
"""

# Instructions

"""

    Initialize the pre-trained GPT-2 tokenizer for gpt2.
    Initialize the pre-trained GPT-2 LMHead model for gpt2.
---

    Encode the seed text to get input tensors using seed_text.
---

    Generate the text by defining the parameters for the input considering a max_length of 100, temperature of 0.7, and an no_repeat_ngram_size of 2 in the GPT-2 model.

"""

# solution

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Initialize the pre-trained model
model = GPT2LMHeadModel.from_pretrained('gpt2')

seed_text = "Once upon a time"

# Encode the seed text to get input tensors
input_ids = tokenizer.encode(seed_text, return_tensors='pt')

# Generate text from the model
output = model.generate(input_ids, max_length=100, temperature=0.7, no_repeat_ngram_size=2, pad_token_id=tokenizer.eos_token_id) 

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

#----------------------------------#

# Conclusion

"""
Great job! You have successfully completed the text completion exercise using the GPT-2 model and tokenizer. By analyzing the generated output, you can explore the fascinating world of language generation and witness the model's ability to generate imaginative and coherent text based on the provided seed text. Well done!
"""

'\n\n'

In [5]:
# exercise 06

"""
Language translation with pretrained PyTorch model

Your team at PyBooks is working on an AI project that involves translation from one language to another. They want to leverage pre-trained models for this task, which can save a lot of training time and resources. The task for this exercise is to set up a translation model from HuggingFace's Transformers library, specifically the T5 (Text-To-Text Transfer Transformer) model, and use it to translate an English phrase to French.

T5Tokenizer, T5ForConditionalGeneration have been loaded for you.
"""

# Instructions

"""

    Initialize the tokenizer and model from the pretrained "t5-small" model.
 
    Encode the input prompt using the tokenizer, making sure to return PyTorch tensors.
 
    Translate the input prompt using model and generate the translated output.

"""

# solution

# Initalize tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

input_prompt = "translate English to French: 'Hello, how are you?'"

# Encode the input prompt using the tokenizer
input_ids = tokenizer.encode(input_prompt, return_tensors="pt")

# Generate the translated ouput
output = model.generate(input_ids, max_length=50)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated text:",generated_text)

#----------------------------------#

# Conclusion

"""
Great job! You've successfully used a pre-trained T5 model to translate an English phrase to French. This example shows the power of transformer models and how they can be leveraged for various NLP tasks, including translation. Well done!
"""

'\n\n'

In [6]:
# exercise 07

"""
Evaluating pretrained text generation model

The PyBooks team has used a pre-trained GPT-2 model that you experimented to generate a text based on a given prompt. Now, they want to evaluate the quality of this generated text. To achieve this, they have tasked you to evaluate generated text using a reference text.

BLEUScore, ROUGEScore have been loaded for you.
"""

# Instructions

"""

    Begin by initializing the two metrics (BLEU and ROUGE) provided from torchmetrics.text.

    Use these initialized metrics to calculate the scores between the generated text and the reference text.

    Display the calculated BLEU and ROUGE scores.

"""

# solution

reference_text = "Once upon a time, there was a little girl who lived in a village near the forest."
generated_text = "Once upon a time, the world was a place of great beauty and great danger. The world of the gods was the place where the great gods were born, and where they were to live."

# Initialize BLEU and ROUGE scorers
bleu = BLEUScore()
rouge = ROUGEScore()

# Calculate the BLEU and ROUGE scores
bleu_score = bleu([generated_text], [[reference_text]])
rouge_score = rouge([generated_text], [[reference_text]])

# Print the BLEU and ROUGE scores
print("BLEU Score:", bleu_score.item())
print("ROUGE Score:", rouge_score)

#----------------------------------#

# Conclusion

"""
Congratulations! You've successfully calculated the BLEU and ROUGE scores for the text generation gpt2 model. Let's analyse the results in the next exercise. Well done!
"""

'\n\n'

# Understanding text generation metrics

At PyBooks, the team just evaluated the performance of a pretrained model using the BLEU score and got a result of approximately 0.082 and a rouge1_fmeasure of around 0.2692. This metric is an indication of precision (how many selected items are relevant) and recall (how many relevant items are selected). How would you interpret this score in terms of the model's performance?

### Possible Answers

The model's generated text is almost identical to the reference text


The rouge measure indicated that model's generated text has some similarity to the reference text{Answers}


The model's generated text bears no similarity to the reference text


The BLUE score indicates that model's generated text has low similarity to the reference text{Answers}